In [14]:
import re
import os, json
import warnings
from pprint import pprint
from textwrap import dedent
from dotenv import load_dotenv
from langchain_core.messages import ToolMessage

warnings.filterwarnings("ignore")
load_dotenv()

True

In [43]:
from langchain_community.tools import TavilySearchResults

query = "최근 한 달 동안 나온 테슬라 CEO와 관련된 영문 기사를 한국어로 요약해줘."

tavily_search = TavilySearchResults(max_results=2)
search_results = tavily_search.invoke(query)
search_results

[{'title': '월~금 업로드] 영어 기사 하루 한 줄 -테슬라 CEO 일론 머스크, 오픈AI ...',
  'url': 'https://www.youtube.com/watch?v=j216YuDJoz4',
  'content': '#영어공부 #매일영어공부 #매일영어 #영어표현 #영어표현배우기 #영단어 #영어기사 #영자신문 #영자신문읽기 #영어단어 #영어독해 #영어기초 #비즈니스영어 #영어뉴스 #원어민영어표현 #미국직장인 #english #영어기사해석#dailyenglish#englishstudy #studyenglish #englishexpressionsa #중급영어 #고급영어 #사회뉴스 #세계뉴스 #미국뉴스 #경제뉴스 #세계경제 #일론머스크 #샘알트먼 #샘알트만 #오픈AI #openAI #인공지능 #AI #benefits #humanity #Microsoft #마이크로소프트 #back #bebacked #bebackedby #backedby #후원받다 #지원받다 #지지하다 #뒷받침하다 #prohibit #prohibitafromb #prohibitfrom #prohibition #금지하다 #못하도록하다 #막다\n\n\n\n\n*기사 출처\nNPR March 1st, 2024 [...] "Elon Musk sues OpenAI for choosing profits over \'the benefit of humanity\'" \nBy Juliana Kim\r\n\r\nhttps://www.npr.org/2024/03/01/1235159084/elon-musk-openai-suit-chatgpt-sam-altman-greg-brockman\n\n*효과음\nCC0 1.0 Universal made by DayDreamSound\nhttps://youtu.be/kfnh9QAfDgA\n\n*이모티콘\n제작자: June Design - Flaticon \n 4 comments [...] [Daily English - 월~금 업로드] 영어 기사 하루 한 줄 -테슬라 

In [44]:
TavilySearchResults # class 살펴보기
type(tavily_search)

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [45]:
tavily_search.name

'tavily_search_results_json'

In [46]:
tavily_search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [47]:
tavily_search.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [48]:
tavily_search.args_schema

langchain_community.tools.tavily_search.tool.TavilyInput

In [49]:
tavily_search.args_schema.model_json_schema()

{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}

In [50]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[tavily_search])

query

'최근 한 달 동안 나온 테슬라 CEO와 관련된 영문 기사를 한국어로 요약해줘.'

In [51]:
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PB8gOihLLPXcm9sYYTI6LIAg', 'function': {'arguments': '{"query":"recent news articles about Tesla CEO"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 103, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BQWj2Ox5jAojVItaenGRu2iHk4io8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e776d12b-b74b-4330-8edd-e4e5e30dff1d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent news articles about Tesla CEO'}, 'id': 'call_PB8gOihLLPXcm9sYYTI6LIAg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 103, 'ou

In [52]:
ai_msg.content

''

In [53]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'recent news articles about Tesla CEO'},
  'id': 'call_PB8gOihLLPXcm9sYYTI6LIAg',
  'type': 'tool_call'}]

In [57]:
# args 스키마 사용
tool_call = ai_msg.tool_calls[0]
tool_output = tavily_search.invoke(tool_call['args'])
pprint(type(tool_output))
pprint(tool_output)

<class 'list'>
[{'content': 'Tesla CEO Elon Musk boards Air Force One in New Jersey. Autos & '
             'Transportationcategory· April 22, 2025. Tesla CEO Elon Musk '
             'dreams big but often delivers late.',
  'score': 0.81347597,
  'title': "Elon Musk News | Today's Latest Stories - Reuters",
  'url': 'https://www.reuters.com/business/elon-musk/'},
 {'content': 'Multi-city event series empowers corporate executives, investors '
             '& tech moguls\n'
             '\n'
             'October 27 - 31, 2025 · Houston, TX\n'
             '\n'
             'Join the Growing Community of Tech Creators and Innovators\n'
             '\n'
             'EXPLORE\n'
             '\n'
             'PROGRAMS\n'
             '\n'
             'MORE\n'
             '\n'
             'join us on  SOCIALS\n'
             '\n'
             'SUBSCRIBE TO NEWSLETTER\n'
             '\n'
             '© 2025  AfroTech ,  Blavity, Inc.  All rights reserved. [...] '
             'Apr 2

In [58]:
# tool_call 사용
tool_message = tavily_search.invoke(tool_call)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content='[{"title": "Elon Musk News | Today\'s Latest Stories - Reuters", "url": "https://www.reuters.com/business/elon-musk/", "content": "Tesla CEO Elon Musk boards Air Force One in New Jersey. Autos & Transportationcategory· April 22, 2025. Tesla CEO Elon Musk dreams big but often delivers late.", "score": 0.81347597}, {"title": "Tesla CEO Elon Musk Admits Full Self-Driving Technology Isn\'t ...", "url": "https://afrotech.com/elon-musk-admits-tesla-full-self-driving-not-ready", "content": "Multi-city event series empowers corporate executives, investors & tech moguls\\n\\nOctober 27 - 31, 2025 · Houston, TX\\n\\nJoin the Growing Community of Tech Creators and Innovators\\n\\nEXPLORE\\n\\nPROGRAMS\\n\\nMORE\\n\\njoin us on  SOCIALS\\n\\nSUBSCRIBE TO NEWSLETTER\\n\\n© 2025  AfroTech ,  Blavity, Inc.  All rights reserved. [...] Apr 25, 2025\\n\\n2 min\\n\\nTesla CEO Elon Musk Admits Full Self-Driving Technology Isn’t Ready 

In [60]:
# tool_output으로 ToolMessage 만들기
from langchain_core.messages.tool import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)
pprint(type(tool_message))
pprint(tool_message)

<class 'langchain_core.messages.tool.ToolMessage'>
ToolMessage(content=[{'title': "Elon Musk News | Today's Latest Stories - Reuters", 'url': 'https://www.reuters.com/business/elon-musk/', 'content': 'Tesla CEO Elon Musk boards Air Force One in New Jersey. Autos & Transportationcategory· April 22, 2025. Tesla CEO Elon Musk dreams big but often delivers late.', 'score': 0.81347597}, {'title': "Tesla CEO Elon Musk Admits Full Self-Driving Technology Isn't ...", 'url': 'https://afrotech.com/elon-musk-admits-tesla-full-self-driving-not-ready', 'content': 'Multi-city event series empowers corporate executives, investors & tech moguls\n\nOctober 27 - 31, 2025 · Houston, TX\n\nJoin the Growing Community of Tech Creators and Innovators\n\nEXPLORE\n\nPROGRAMS\n\nMORE\n\njoin us on  SOCIALS\n\nSUBSCRIBE TO NEWSLETTER\n\n© 2025  AfroTech ,  Blavity, Inc.  All rights reserved. [...] Apr 25, 2025\n\n2 min\n\nTesla CEO Elon Musk Admits Full Self-Driving Technology Isn’t Ready Even After Customers Pa